In [2]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 972.3 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [131]:
import random
from uuid import uuid4
from pprint import pprint
from elasticsearch import Elasticsearch
import elasticsearch.helpers as helpers

In [132]:
es = Elasticsearch(hosts=['https://es01:9200'], verify_certs=False, http_auth=('elastic', 'password'))

/root/miniconda3/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://es01:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/tmp/ipykernel_446/2163348175.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=['https://es01:9200'], verify_certs=False, http_auth=('elastic', 'password'))


In [154]:
from elasticsearch import Elasticsearch, exceptions
index = 'catalog'
try:
    es.indices.delete(index=index)
except exceptions.NotFoundError as e:
    pass

/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [163]:
from pprint import pprint
print(es.indices.get(index=index))

{'catalog': {'aliases': {}, 'mappings': {'properties': {'column_description': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 1024}}}, 'column_description_embedding': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'l2_norm'}, 'column_name': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'table_catalog': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'table_description': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 1024}}}, 'table_description_embedding': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'l2_norm'}, 'table_name': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'table_schema': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shar

/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [137]:
import pandas as pd
import ast
dataset = pd.read_csv('sample_catalog_annotations.csv', usecols=['table_hash', 'table_catalog', 'table_schema', 'table_name', 'columns', 'table_description', 'column_descriptions'])
dataset['column_description'] = dataset['column_descriptions'].apply(ast.literal_eval)
dataset = dataset.drop('column_descriptions', axis=1).explode('column_description')
dataset['column_name'] = dataset['column_description'].apply(lambda x: x[0] if x else None)
dataset['column_description'] = dataset['column_description'].apply(lambda x: x[1] if x else None)
dataset['id'] = dataset.reset_index().index
dataset.set_index('id', inplace=True)
dataset

table_hash table_catalog table_schema  \
id                                                                 
0    b59fec39143ab6ec908fba0edc5d2677     sqlserver          dbo   
1    b59fec39143ab6ec908fba0edc5d2677     sqlserver          dbo   
2    b59fec39143ab6ec908fba0edc5d2677     sqlserver          dbo   
3    31fcf40fd7bd3e0fe96bb7a4c20388d9     sqlserver          dbo   
4    31fcf40fd7bd3e0fe96bb7a4c20388d9     sqlserver          dbo   
..                                ...           ...          ...   
122  70e6774d16ae5b400c276f3939f6b9eb       uploads       public   
123  70e6774d16ae5b400c276f3939f6b9eb       uploads       public   
124  59b92905505aa81bef19b0c259c2e232       uploads       public   
125  59b92905505aa81bef19b0c259c2e232       uploads       public   
126  59b92905505aa81bef19b0c259c2e232       uploads       public   

              table_name                                            columns  \
id                                                                            
0              aux_table  {mt_key1:varchar(128),mt_key2:bigint,mt_commen...   
1              aux_table  {mt_key1:varchar(128),mt_key2:bigint,mt_commen...   
2              aux_table  {mt_key1:varchar(128),mt_key2:bigint,mt_commen...   
3         bucketing_cols  {sd_id:bigint,bucket_col_name:varchar(255),int...   
4         bucketing_cols  {sd_id:bigint,bucket_col_name:varchar(255),int...   
..                   ...                                                ...   
122          htsdata (1)  {"hts number:varchar",indent:bigint,descriptio...   
123          htsdata (1)  {"hts number:varchar",indent:bigint,descriptio...   
124  new_tariff_list_max  {time_of_tariff:varchar,hs6:bigint,tariff:double}   
125  new_tariff_list_max  {time_of_tariff:varchar,hs6:bigint,tariff:double}   
126  new_tariff_list_max  {time_of_tariff:varchar,hs6:bigint,tariff:double}   

                                     table_description  \
id                                                       
0    The aux_table stores metadata for efficient da...   
1    The aux_table stores metadata for efficient da...   
2    The aux_table stores metadata for efficient da...   
3    This table stores data related to buckets and ...   
4    This table stores data related to buckets and ...   
..                                                 ...   
122  The htsdata table contains data related to cus...   
123  The htsdata table contains data related to cus...   
124  The new_tariff_list_max table contains informa...   
125  The new_tariff_list_max table contains informa...   
126  The new_tariff_list_max table contains informa...   

                                    column_description  \
id                                                       
0    The mt_key1 column holds 128-character unique ...   
1    The mt_key2 column represents a large numerica...   
2    The mt_comment column holds brief descriptions...   
3    The sd_id is a unique identifier assigned to e...   
4    The bucket_col_name column represents the name...   
..                                                 ...   
122  The "quota quantity" column represents the max...   
123  The column "additional duties" represents extr...   
124  The column "time_of_tariff" represents the spe...   
125  The column 'hs6' represents the unique identif...   
126  The tariff column represents the maximum value...   

                      column_name  
id                                 
0            mt_key1:varchar(128)  
1                  mt_key2:bigint  
2         mt_comment:varchar(255)  
3                    sd_id:bigint  
4    bucket_col_name:varchar(255)  
..                            ...  
122         quota quantity:double  
123      additional duties:double  
124        time_of_tariff:varchar  
125                    hs6:bigint  
126                 tariff:double  

[127 rows x 8 columns]

In [138]:
# define index name
INDEX_NAME = index

# flag to check if index has to be deleted before creating
SHOULD_DELETE_INDEX = True

# define index mapping
INDEX_MAPPING = {
    "properties": {
        "table_catalog": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
        },
        "table_schema": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
        },
        "table_name": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
        },
        "column_name": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
        },
        "table_description": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 1024}},
        },
        "column_description": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
        },
        "table_description_embedding": {
            "type": "dense_vector",
            "dims": 384,
            "index": True,
            "similarity": "l2_norm",
        },
        "column_description_embedding": {
            "type": "dense_vector",
            "dims": 384,
            "index": True,
            "similarity": "l2_norm",
        },
    }
}

INDEX_SETTINGS = {
    "index": {
        "number_of_replicas": "1",
        "number_of_shards": "1",
    }
}

# check if we want to delete index before creating the index
if SHOULD_DELETE_INDEX:
    if es.indices.exists(index=INDEX_NAME):
        print("Deleting existing %s" % INDEX_NAME)
        es.indices.delete(index=INDEX_NAME, ignore=[400, 404])

print("Creating index %s" % INDEX_NAME)
es.indices.create(
    index=INDEX_NAME, mappings=INDEX_MAPPING, settings=INDEX_SETTINGS, ignore=[400, 404]
)
actions = []
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
rows = dataset.copy()
rows["table_description_embedding"] = model.encode(rows["table_description"].fillna('').astype(str)).tolist()
rows["column_description_embedding"] = model.encode(rows["column_description"].fillna('').astype(str)).tolist()
for idx, row in rows.iterrows():
    actions.append({"index": {"_index": INDEX_NAME}})
    actions.append(dict(row))
es.bulk(index=INDEX_NAME, operations=actions)

/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/tmp/ipykernel_446/4282608558.py:60: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=INDEX_NAME, ignore=[400, 404])
/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Deleting existing catalog
Creating index catalog


/tmp/ipykernel_446/4282608558.py:63: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(
/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'errors': False, 'took': 402, 'items': [{'index': {'_index': 'catalog', '_id': '7RMtNo4Bk7K-iX_fCnJ_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'catalog', '_id': '7hMtNo4Bk7K-iX_fCnJ_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'catalog', '_id': '7xMtNo4Bk7K-iX_fCnJ_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'catalog', '_id': '8BMtNo4Bk7K-iX_fCnJ_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'catalog', '_id': '8RMtNo4Bk7K-iX_fCnJ_', '_version': 1, 'result': 'created', '_shards': {'total': 2

In [176]:
def search(index, query, n=10):
    INDEX_NAME = index
    vector = model.encode([query])[0]
    results = es.search(
        index=index,
        knn=[
            {
                "field": "HierarchicDescription_embedding",
                "query_vector": vector,
                "num_candidates": 10*n,
                "k": 4*n,
            }
        ],
        size=n,
    )
    
    
    def show_results(results):
        def _assimilate_score(result):
            result_dict = result["_source"]
            result_dict.update({"score": result["_score"]})
            result_dict.update({"id": result["_id"]})
            return result_dict
        return pd.DataFrame.from_dict([_assimilate_score(result) for result in results])
    return show_results(results["hits"]["hits"])

search("hs4", "milk", 10) #.groupby(['table_catalog', 'table_schema', 'table_name']).agg({'score': sum, 'column_name': list}).sort_values(by='score', ascending=False)

/root/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Code                              HierarchicDescription  \
0  8434               milking machines and dairy machinery   
1   402  milk and cream concentrated or containing adde...   
2   405  butter and other fats and oils derived from mi...   
3   401  milk and cream not concentrated nor containing...   
4   905                                            vanilla   
5  5011                                               lard   
6  1004                                            >> oats   
7   404  whey whether or not concentrated or containing...   
8  1003                                          >> barley   
9   403  kephir and other fermented or acidified milk a...   

                     HierarchicDescription_embedding     score   id  
0  [-0.037625253200531006, -0.09057734161615372, ...  0.807582  935  
1  [0.03967500850558281, -0.09143534302711487, 0....  0.778820   26  
2  [0.003921830095350742, -0.09914423525333405, -...  0.762286   29  
3  [0.07775484025478363, -0.12136857956647873, 0....  0.749710   25  
4  [0.016470177099108696, -0.08789700269699097, 0...  0.719206   78  
5  [0.033213771879673004, -0.08884340524673462, -...  0.716267  551  
6  [-0.015581049956381321, 0.030825864523649216, ...  0.714140   88  
7  [0.014100012369453907, -0.10721537470817566, 0...  0.714024   28  
8  [-0.014625953510403633, 0.015427705831825733, ...  0.694697   87  
9  [0.011933179572224617, -0.07479555904865265, 0...  0.694554   27